## Stockholms stads offentliga toaletter
* [#1](https://github.com/salgo60/Offentliga-toaletter/issues/1)

---- 
0.1 skapade
0.2 la till 

In [1]:
import time
from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))



Start: 2026-01-22 17:19:13


In [2]:
SthlmfileName ="Toalett_Punkt_Sthlm_20260123.gpkg"

In [3]:
import geopandas as gpd

# Läs in geopackage-filen
gdf = gpd.read_file(SthlmfileName)

# Visa grundläggande information
print(gdf.head())
print(gdf.crs)
print(gdf.columns)

                           Adress Anläggningsnummer  Antal_WC  \
0                Frödingsvägen 13               G84       NaN   
1      Norr Mälarstrand 3 - kajen               G60       NaN   
2                   Fjällgatan 37               G85       NaN   
3                   Sveavägen 39A               G67       NaN   
4  Norra Bantorget - Vasagatan 59               G14       1.0   

   Antal_urinoarer                                        Beskrivning  Bild  \
0              NaN                                  Inbyggd i byggnad  None   
1              NaN  Tillgänglighetsanpassad WC, 2 x WC och 1 x uri...  None   
2              NaN                   Inbyggd endast i drift sommartid  None   
3              NaN                            Tillgänglighetsanpassad  None   
4              NaN                         Tillgänglighetsanpassad WC  None   

    CHANGE_DATE           CID   CREATE_DATE            Driftansvar  ...  \
0  2.025021e+11  1.659929e+15  2.019021e+11          Miljöt

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: GPKG: unrecognized user_version=0x00000000 (0) on 'Toalett_Punkt_Sthlm_20260123.gpkg'
  return ogr_read(


In [4]:
def convert_misread_date(x):
    try:
        if pd.isna(x):
            return None
        # gör till int, ta bara siffrorna
        s = str(int(float(x)))
        # om 8 tecken → YYYYMMDD
        if len(s) == 8:
            return pd.to_datetime(s, format="%Y%m%d", errors="coerce")
        return None
    except Exception:
        return None

for col in [c for c in gdf.columns if "DATE" in c.upper()]:
    gdf[col + "_fmt"] = gdf[col].apply(convert_misread_date)

In [5]:
import geopandas as gpd
import pandas as pd

# Läs in
gdf = gpd.read_file(SthlmfileName)

# Beräkna centroid i meter-CRS (SWEREF99 TM) och konvertera till WGS84
gdf_proj = gdf.to_crs(epsg=3006)
gdf["geometry_centroid"] = gdf_proj.geometry.centroid.to_crs(epsg=4326)

# Lägg till lat/lon
gdf["lon"] = gdf["geometry_centroid"].x
gdf["lat"] = gdf["geometry_centroid"].y

# Lista möjliga datumkolumner
date_cols = [c for c in gdf.columns if "DATE" in c.upper() or "DATUM" in c.upper()]
print("🕒 Datumkolumner:", date_cols)

# Funktion för att försöka tolka stora siffror som tidsstämplar
def convert_date(x):
    try:
        # Om NaN
        if pd.isna(x):
            return None
        # Om redan datetime
        if isinstance(x, pd.Timestamp):
            return x
        # Om det ser ut som 1.738443e+15 (millisekunder)
        x = float(x)
        if x > 1e12:  # troligen millisekunder
            return pd.to_datetime(x, unit="ms")
        elif x > 1e9:  # sekunder
            return pd.to_datetime(x, unit="s")
        else:
            return None
    except Exception:
        return None

# Konvertera alla identifierade datumfält
for col in date_cols:
    gdf[col + "_fmt"] = gdf[col].apply(convert_date)

# Visa exempel på konverterade datum
print(gdf[[c for c in gdf.columns if "DATE" in c or "fmt" in c]].head())

gdf.drop(columns=["geometry", "geometry_centroid"]).to_csv("SthlmfileName_Toalett_fix.csv", index=False)
print("✅ Sparade 'SthlmfileName_Toalett_fix.csv' med riktiga datum och koordinater")

🕒 Datumkolumner: ['CHANGE_DATE', 'CREATE_DATE']
    CHANGE_DATE   CREATE_DATE CHANGE_DATE_fmt CREATE_DATE_fmt
0  2.025021e+11  2.019021e+11            None            None
1  2.024012e+11  2.018023e+11            None            None
2  2.022091e+11  2.019021e+11            None            None
3  2.024012e+11  2.018023e+11            None            None
4  2.024012e+11  2.018011e+11            None            None
✅ Sparade 'SthlmfileName_Toalett_fix.csv' med riktiga datum och koordinater


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: GPKG: unrecognized user_version=0x00000000 (0) on 'Toalett_Punkt_Sthlm_20260123.gpkg'
  return ogr_read(


In [6]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 42 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   Adress                      101 non-null    object  
 1   Anläggningsnummer           101 non-null    object  
 2   Antal_WC                    4 non-null      float64 
 3   Antal_urinoarer             2 non-null      float64 
 4   Beskrivning                 75 non-null     object  
 5   Bild                        1 non-null      object  
 6   CHANGE_DATE                 101 non-null    float64 
 7   CID                         101 non-null    float64 
 8   CREATE_DATE                 101 non-null    float64 
 9   Driftansvar                 101 non-null    object  
 10  El_Service                  0 non-null      object  
 11  FEATURE_TYPE_OBJECT_ID      101 non-null    int64   
 12  FEATURE_TYPE_VERSION_ID     101 non-null    int16   
 13  Fabrikat    

In [7]:
gdf.describe()

,Antal_WC,Antal_urinoarer,CHANGE_DATE,CID,CREATE_DATE,FEATURE_TYPE_OBJECT_ID,FEATURE_TYPE_VERSION_ID,OBJECT_ID,VALID_FROM,VERSION_ID,fid,lon,lat
count,4.0,2.0,1.010000e+02,1.010000e+02,1.010000e+02,101.0,101.0,1.010000e+02,1.010000e+02,101.0,101.000000,101.000000,101.000000
mean,1.0,1.0,2.024175e+11,1.747852e+15,2.019862e+11,15303196.0,1.0,1.747852e+07,2.019861e+11,1.0,51.000000,18.028833,59.323680
std,0.0,0.0,7.942318e+07,3.398323e+14,2.930167e+08,0.0,0.0,3.398322e+06,2.930255e+08,0.0,29.300171,0.064846,0.032642
min,1.0,1.0,2.022090e+11,1.530321e+15,2.018011e+11,15303196.0,1.0,1.530321e+07,2.018011e+11,1.0,1.000000,17.843198,59.243971
25%,1.0,1.0,2.024012e+11,1.530326e+15,2.018011e+11,15303196.0,1.0,1.530326e+07,2.018011e+11,1.0,26.000000,18.002871,59.301593
50%,1.0,1.0,2.024012e+11,1.538074e+15,2.018023e+11,15303196.0,1.0,1.538074e+07,2.018022e+11,1.0,51.000000,18.052149,59.326010
75%,1.0,1.0,2.025101e+11,1.775757e+15,2.020092e+11,15303196.0,1.0,1.775757e+07,2.020092e+11,1.0,76.000000,18.075276,59.339646
max,1.0,1.0,2.025102e+11,2.358270e+15,2.025101e+11,15303196.0,1.0,2.358270e+07,2.025101e+11,1.0,101.000000,18.130955,59.413180


In [8]:
from shapely.geometry import base

for col in gdf.columns:
    has_geom = gdf[col].apply(lambda x: isinstance(x, base.BaseGeometry)).any()
    if has_geom:
        print(f"🧱 Kolumn '{col}' innehåller geometrier")


🧱 Kolumn 'geometry' innehåller geometrier
🧱 Kolumn 'geometry_centroid' innehåller geometrier


In [9]:
from urllib.parse import quote

def build_toilet_popup(row):
    lat = row["lat"]
    lon = row["lon"]

    name = "Offentlig toalett"
    address = row.get("Adress", "")
    status = row.get("Status", "")
    operator = row.get("Driftansvar", "Stockholms stad")
    accessible = row.get("Tillgänglighetsanpassad", "")
    facility_id = row.get("Anläggningsnummer", "")

    # --- Tillgänglighet ---
    wheelchair_html = {
        "Ja": "♿ <b>Tillgänglig:</b> Ja ✅<br>",
        "Nej": "♿ <b>Tillgänglig:</b> Nej ❌<br>",
    }.get(accessible, "♿ <b>Tillgänglig:</b> Okänd ❔<br>")

    # --- OSM Note ---
    note_text = (
        f"Kommunal toalett enligt Stockholms stad. "
        f"Adress: {address}. "
        f"Tillgänglig: {accessible}. "
        f"Anläggningsnummer: {facility_id}."
    )
    note_url = (
        f"https://www.openstreetmap.org/note/new"
        f"#map=18/{lat}/{lon}&text={quote(note_text)}"
    )

    # --- App / editor-länkar ---
    gomap_js = (
        f"window.location='gomap!!://edit?lat={lat}&lon={lon}&zoom=18';"
        f"setTimeout(function(){{window.open('https://www.openstreetmap.org/edit#map=18/{lat}/{lon}');}},1000);"
        f"return false;"
    )

    menu_html = f"""
    <div style='margin-top:8px; text-align:center;'>
      <a href='#' onclick="{gomap_js}"
         style='background:#ff6600;color:white;padding:6px 10px;
                border-radius:6px;text-decoration:none;margin:2px;
                display:inline-block;font-size:13px;'>
         🛠️ Go Map!!
      </a>
      <a href='{note_url}' target='_blank'
         style='background:#fbbc04;color:black;padding:6px 10px;
                border-radius:6px;text-decoration:none;margin:2px;
                display:inline-block;font-size:13px;'>
         📝 OSM Note
      </a>
    </div>
    """

    tools_html = f"""
    <details style='margin-top:6px; text-align:center;'>
      <summary style='cursor:pointer;font-weight:bold;'>🔗 Fler kartverktyg</summary>
      <a href='https://www.openstreetmap.org/edit#map=18/{lat}/{lon}' target='_blank'>✏️ iD Editor</a><br>
      <a href='https://level0.osmz.ru/?url=https://www.openstreetmap.org/?mlat={lat}&mlon={lon}' target='_blank'>🧮 Level0</a><br>
      <a href='https://www.mapillary.com/app/?lat={lat}&lng={lon}&z=17' target='_blank'>🟢 Mapillary</a><br>
      <a href='https://www.panoramax.xyz/map/#17/{lat}/{lon}' target='_blank'>📸 Panoramax</a><br>
      <a href='https://overpass-turbo.eu/map.html?zoom=17&lat={lat}&lon={lon}' target='_blank'>🧩 Overpass Turbo</a><br>
    </details>
    """

    popup_html = f"""
    <div style='font-family:Arial; font-size:13px; max-width:320px;'>
      <b>🚻 {name}</b><br>
      <b>Adress:</b> {address}<br>
      <b>Status:</b> {status}<br>
      <b>Driftansvar:</b> {operator}<br>
      <b>Anläggningsnr:</b> {facility_id}<br>
      {wheelchair_html}
      📍 <a href='https://www.openstreetmap.org/?mlat={lat}&mlon={lon}#map=18/{lat}/{lon}'
            target='_blank'>{lat:.5f}, {lon:.5f}</a><br>

      {menu_html}
      {tools_html}

      <hr style="margin:6px 0;">
      <small>
        <b>Källa:</b>
        <a href="https://dataportalen.stockholm.se/dataportalen/GetMetaDataById?id=LvFeature15303196&showmetadataview"
           target="_blank">
           Stockholms stads öppna data
        </a>
      </small>
    </div>
    """
    return popup_html


In [10]:
import folium 
m = folium.Map(location=[59.3293, 18.0686], zoom_start=11)

gdf.to_csv("Sthlm.csv")

In [11]:
from shapely.geometry import base

# Ta bort alla geometrier utom den huvudsakliga
gdf_clean = gdf.copy()

# ta bort "geometry_centroid" och andra geometrikolumner
geom_cols = [c for c in gdf_clean.columns 
             if c != "geometry" and gdf_clean[c].apply(lambda x: isinstance(x, base.BaseGeometry)).any()]
if geom_cols:
    print("🧹 Tar bort extra geometri-kolumner:", geom_cols)
    gdf_clean = gdf_clean.drop(columns=geom_cols, errors="ignore")

# Skapa Folium-lager utan att försöka serialisera själva geometry-kolumnen
stad_layer = folium.FeatureGroup(name="🟠 Toaletter (Stockholms stad)")

for _, row in gdf_clean.iterrows():
    if pd.isna(row["lat"]) or pd.isna(row["lon"]):
        continue

    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=8,
        fill=True,
        fill_color="#ff6600",
        color="#cc5500",
        weight=2,
        fill_opacity=0.7,
        popup=folium.Popup(build_toilet_popup(row), max_width=350),
    ).add_to(stad_layer)


stad_layer.add_to(m)


🧹 Tar bort extra geometri-kolumner: ['geometry_centroid']


In [12]:
from datetime import datetime, timezone

generated_at = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M UTC")


In [13]:


legend_html = f"""
<div style="
    position: fixed;
    bottom: 30px;
    left: 30px;
    width: 320px;
    z-index: 10000;
    background: white;
    padding: 10px;
    border: 2px solid #ccc;
    border-radius: 6px;
    font-size: 12px;
">
<b>🟠 Offentliga toaletter – Stockholms stad version 0.2</b><br><br>
<small><i>Karta genererad: {generated_at}</i></small><br><br>

<b>Syfte</b><br>
Datat används för att verifiera och förbättra OSM Stockholm stad gör om en hel del se issue
(<i>amenity=toilets</i>).<br><br>

<b>Mer info</b><br>
• <a href="https://github.com/salgo60/Offentliga-toaletter/issues/1" target="_blank">
GitHub Issue #1
</a><br>
• <a href="https://dataportalen.stockholm.se/dataportalen/GetMetaDataById?id=LvFeature15303196&showmetadataview" target="_blank">
Stockholms stads öppna data
</a>
</div>
"""



In [14]:
m.get_root().html.add_child(folium.Element(legend_html))
folium.LayerControl(collapsed=False).add_to(m)

In [15]:
m

In [20]:
# Exportera kartan https://salgo60.github.io/Offentliga-toaletter/Notebook/Toaletter_sthlm.html
m.save("Toaletter_sthlm_02.html") 
m.save("Toaletter_sthlm_latest.html") 

In [19]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

Date: 2026-01-22 17:22:42
Total time elapsed: 209.16 seconds
